In [1]:
import pandas as pd

In [2]:
sarcasm_headlines = pd.read_json("../data/sarcasm-headlines.json", lines=True)
sarcasm_headlines.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
sarcasm_headlines['is_sarcastic'].value_counts()

is_sarcastic
0    14985
1    11724
Name: count, dtype: int64

In [6]:
sarcastic_headlines = sarcasm_headlines[sarcasm_headlines['is_sarcastic'] == 1]
sarcastic_headlines.shape

(11724, 3)

In [7]:
not_sarcastic_headlines = sarcasm_headlines[sarcasm_headlines['is_sarcastic'] == 0]
not_sarcastic_headlines.shape

(14985, 3)

In [8]:
not_sarcastic_headlines = not_sarcastic_headlines.sample(sarcastic_headlines.shape[0])
not_sarcastic_headlines.shape

(11724, 3)

In [11]:
sarcasm_headlines = pd.concat([sarcastic_headlines, not_sarcastic_headlines])
sarcasm_headlines.shape

(23448, 3)

In [10]:
sarcasm_headlines['is_sarcastic'].value_counts()

is_sarcastic
1    11724
0    11724
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = sarcasm_headlines['headline']
y = sarcasm_headlines['is_sarcastic']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=sarcasm_headlines['is_sarcastic'])

In [15]:
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/2"

In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

C:\Users\bekasker\miniconda3\envs\python3.10\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [18]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(preprocessor_url)
encoder_inputs = preprocessor(text_input)

In [27]:
encoder = hub.KerasLayer(encoder_url)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

In [28]:
layer = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)
layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

In [29]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [30]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=1)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import numpy as np

In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()
y_pred = np.where(y_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred) 

In [ ]:
import matplotlit.pyplot as plt
import seaborn as sns

In [ ]:
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))